<a href="https://colab.research.google.com/github/JasonTJH/mycolab/blob/main/VideoToPPT_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video To PPT/PDF

In [ ]:
# Installing dependencies
!pip install av
!pip install img2pdf
!pip install python-pptx

# Importing dependencies
import av
import av.datasets
import os
from PIL import Image
import subprocess
import itertools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import time
from hashlib import md5
from scipy.spatial import distance
import img2pdf
import pptx
import pptx.util
import glob
from imageio import imread
from google.colab import files

     |████████████████████████████████| 37.2MB 1.3MB/s 
     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 9.3MB 5.5MB/s 
     |████████████████████████████████| 153kB 37.2MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.19-cp37-none-any.whl size=469953 sha256=f35a930b8fc6bf879badcd627cd01eacd5bfaec75c0c96d182452fd3f74cce66
  Stored in directory: /root/.cache/pip/wheels/94/ef/02/9357c6781fbe3fee0e5e04bad23904d096e39d420423519631
Successfully built python-pptx


In [ ]:
### INPUTS
### remove any unwanted jpg files in the current working directory

vidpath = "/content/drive/MyDrive/Colab_Notebooks/REPLACE_WITH_VIDEO_NAME.mp4"
box_x = int(1570) ##this + width should be equal total width of video resolution (1920)
box_y = int(800)  ##this + height should be equal total height of video resolution (1080)
box_w = int(350)  ## WIDTH OF BOX
box_h = int(280)  ## HEIGHT OF BOX
imgfreq = int(5)  #in seconds - only for detailed raw images
zeropad = int(5)  #for formatting temp file names, max image count is 5 digits

## Getting key JPG images from video

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Fastest but might miss out on some stuff #1000fpm
def mp4tojpg(vidpath,type): #vidpath & blackvideo.mp4    
  content = av.datasets.curated(vidpath)
  with av.open(content) as container:
      # Signal that we only want to look at keyframes.
      #container.streams.video[0].thread_type = 'AUTO' #auto threading
      stream = container.streams.video[0]
      stream.codec_context.skip_frame = type # SkipType Names - NONE, DEFAULT, NONREF, NONINTRA, NONKEY, ALL

      for frame in container.decode(stream):
          #print(frame)
          # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
          frame.to_image().save(
              #'{:04d}.jpg'.format(frame.index),
              f'{str(frame.index).zfill(zeropad)}.jpg',
              quality=50)
      return os.listdir()

## Combining JPG to new video (30fps) to be processed

In [ ]:
'''
#Faster but poor image quality
def jpgtomp4(output): #'newvideo.mp4'
  output = av.open(output, 'w')
  stream = output.add_stream('libx264rgb', 30) #libx264 or libx264rgb
  stream.pix_fmt = "rgb24" #yuv420p or rgb24 
  #stream.options["crf"] = "0"
  for image in sorted([file for file in os.listdir() if file.endswith(('jpg'))]):
    frame = av.VideoFrame.from_image(Image.open(image))
    packet = stream.encode(frame)
    output.mux(packet)
  output.mux(stream.encode())
  output.close()
'''
# Slower but probably a bit btr quality
#!ffmpeg -framerate 30 -pattern_type glob -i '*.jpg' -c:v libx264 -pix_fmt yuv420p newvideo.mp4
def jpgtomp42(output): #newvideo.mp4, blackvideo.mp4
  subprocess.run(['ffmpeg', '-framerate', '30', '-pattern_type', 'glob', '-i', '*.jpg', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output])


In [ ]:
#removes all jpg files to avoid clutter
!rm *.jpg

rm: cannot remove '*.jpg': No such file or directory


## Remove instructor face on new video

In [ ]:
#!ffmpeg -y -i newvideo.mp4 -vf "drawbox=x=1570:y=800:w=350:h=280:color=black@1:t=max" blackvideo.mp4 #4 min execution

def addbox(input,output,box_x,box_y,box_w,box_h): #newvideo.mp4, blackvideo.mp4
  subprocess.run(['ffmpeg', '-y', '-i', input, '-vf', 'drawbox=x='+str(box_x)+':y='+str(box_y)+':w='+str(box_w)+':h='+str(box_h)+':color=black@1:t=max', output],check=True)

#!ffmpeg -i blackvideo.mp4 -vf mpdecimate,setpts=N/FRAME_RATE/TB cleanvideo.mp4 #7 min execution

## Using Hamming Distance to find and remove similar images

In [ ]:
#Hamming distance

def hamming_distance(image, image2):
    score = distance.hamming(image, image2)
    return score

def resize(image, height=30, width=30):
    row_res = cv2.resize(image,(height, width), interpolation = cv2.INTER_AREA).flatten()
    col_res = cv2.resize(image,(height, width), interpolation = cv2.INTER_AREA).flatten('F')
    return row_res, col_res

def img_gray(image):
    image = plt.imread(image)
    return np.average(image, weights=[0.299, 0.587, 0.114], axis=2)

def difference_score(image, height = 30, width = 30):
    gray = img_gray(image)
    row_res, col_res = resize(gray, height, width)
    difference = intensity_diff(row_res, col_res)
    return difference

def intensity_diff(row_res, col_res):
    difference_row = np.diff(row_res)
    difference_col = np.diff(col_res)
    difference_row = difference_row > 0
    difference_col = difference_col > 0
    return np.vstack((difference_row, difference_col)).flatten()
    #return difference_row
    #return np.vstack((difference_row, difference_col)) #str method

def difference_score_dict(image_list):
    ds_dict = {}
    duplicates = []
    for image in image_list:
        ds = difference_score(image)
        
        if image not in ds_dict:
            ds_dict[image] = ds
        else:
            duplicates.append((image, ds_dict[image]) )
    
    return  duplicates, ds_dict

def get_duplicates():
  image_files = [file for file in os.listdir() if file.endswith(('jpg'))]
  duplicates, ds_dict = difference_score_dict(image_files)
  
  for k1,k2 in itertools.combinations(ds_dict, 2):
    if hamming_distance(ds_dict[k1], ds_dict[k2])< .10:
        duplicates.append((k1,k2))
  dupes = [x[1] for x in duplicates]
  dupes = list(set(dupes))
  return dupes

def remove_duplicates(dupes):
  for i in dupes:
    os.remove(i)

def show_duplicates(duplicates):
  for file_names in duplicates[:100]:
    try:
        plt.subplot(121),plt.imshow(plt.imread(file_names[0]))
        plt.title('Duplicate'), plt.xticks([]), plt.yticks([])

        plt.subplot(122),plt.imshow(plt.imread(file_names[1]))
        plt.title('Original'), plt.xticks([]), plt.yticks([])
        plt.show()
    
    except OSError as e:
        continue

## Compressing JPG images

In [ ]:
def compress_jpg():
  images = [file for file in os.listdir() if file.endswith(('jpg'))]
  for image in sorted(images):
    # 1. Open the image
    img = Image.open(image)
    # 2. Compressing the image
    img.save("Cmp"+image,
             optimize=True,
             quality=30)

In [ ]:
def remove_uncompressed_jpg():
  for i in range(0, 10000):
    try:
      os.remove(str(i).zfill(zeropad) + ".jpg")
    except:
      pass

## Combining compressed slides to PDF (output.pdf)

In [ ]:

"""
with open("output.pdf", "wb") as f:
    f.write(img2pdf.convert([i for i in os.listdir('.') if i.endswith(".jpg")]))
"""
def make_pdf(output): # 'output.pdf'
	# convert all files ending in .jpg inside a directory
	dirname = "/content"
	with open(output,"wb") as f:
		imgs = []
		for fname in os.listdir(dirname):
			if not fname.endswith(".jpg"):
				continue
			path = os.path.join(dirname, fname)
			if os.path.isdir(path):
				continue
			imgs.append(path)
		f.write(img2pdf.convert(sorted(imgs)))

##Combining compressed slides to PPT (output.ppt)


In [ ]:
##
##  Copy & Paste Tool for images to PowerPoint(.pptx)
##
def make_ppt(output): #'output.pptx'
  OUTPUT_TAG = output

  # new
  prs = pptx.Presentation()
  # open
  # prs_exists = pptx.Presentation("some_presentation.pptx")

  # default slide width
  prs.slide_width = 9144000
  # slide height @ 4:3
  prs.slide_height = 6858000
  # slide height @ 16:9
  #prs.slide_height = 5143500

  # title slide
  slide = prs.slides.add_slide(prs.slide_layouts[0])
  # blank slide
  #slide = prs.slides.add_slide(prs.slide_layouts[6])

  # set title
  title = slide.shapes.title
  title.text = OUTPUT_TAG

  pic_left  = int(prs.slide_width * 0)
  pic_top   = int(prs.slide_height * 0.1)
  pic_width = int(prs.slide_width * 1)

  for g in sorted(glob.glob('*.jpg')):
      #print (g)
      slide = prs.slides.add_slide(prs.slide_layouts[6])

      #tb = slide.shapes.add_textbox(0, 0, prs.slide_width, pic_top / 2)
      #p = tb.text_frame.add_paragraph()
      #p.text = g
      #p.font.size = pptx.util.Pt(14)

      img = imread(g)
      pic_height = int(pic_width * img.shape[0] / img.shape[1])
      #pic   = slide.shapes.add_picture(g, pic_left, pic_top)
      pic   = slide.shapes.add_picture(g, pic_left, pic_top, pic_width, pic_height)

  prs.save("%s" % OUTPUT_TAG)

In [ ]:
def downloadfiles():
  files.download('output.pptx')
  files.download('output.pdf')
  #files.download('newvideo.mp4')
  #files.download('blackvideo.mp4')

In [ ]:
#!rm -rf /content/test_frames1

In [ ]:
!rm *.jpg

rm: cannot remove '*.jpg': No such file or directory



# MAIN

In [ ]:
def main(vidpath,box_x,box_y,box_w,box_h):
  start = time.time()

  mp4tojpg(vidpath, 'NONINTRA') # SkipType Names - NONE, DEFAULT, NONREF, NONINTRA, NONKEY, ALL (>>> Skips more)
  print("Key images extracted in " + str(time.time()-start) + " seconds")
  jpgtomp42('newvideo.mp4')
  print("New video created in " + str(time.time()-start) + " seconds")
  addbox('newvideo.mp4','blackvideo.mp4',box_x,box_y,box_w,box_h)
  print("Black video created in " + str(time.time()-start) + " seconds")
  mp4tojpg('/content/blackvideo.mp4','NONREF') # SkipType Names - NONE, DEFAULT, NONREF, NONINTRA, NONKEY, ALL (>>> Skips more)
  print("Key images extracted in " + str(time.time()-start) + " seconds")
  remove_duplicates(get_duplicates())
  print("Duplicates removed in " + str(time.time()-start) + " seconds")
  #show_duplicates(difference_score_dict()[0])
  compress_jpg()
  print("JPG compressed in " + str(time.time()-start) + " seconds")
  remove_uncompressed_jpg()
  print("Uncompressed JPG removed in " + str(time.time()-start) + " seconds")
  make_pdf('output.pdf')
  make_ppt('output.pptx')
  print("PDF/PPT done in " + str(time.time()-start) + " seconds")

In [ ]:
main(vidpath, box_x, box_y, box_w, box_h)

illegal short term buffer state detected
illegal short term buffer state detected
 (repeated 9 more times)


Key images extracted in 284.7879681587219 seconds
New video created in 560.4427454471588 seconds
Black video created in 791.6722428798676 seconds
Key images extracted in 939.2653307914734 seconds
Duplicates removed in 1555.807375907898 seconds
JPG compressed in 1598.003602027893 seconds
Uncompressed JPG removed in 1598.1017999649048 seconds
PDF/PPT done in 1629.4089951515198 seconds


In [ ]:
!rm *.jpg
'''
!rm *.mp4
!rm *.pptx
!rm *.pdf
'''

'\n!rm *.mp4\n!rm *.pptx\n!rm *.pdf\n'